<div align="center">
<h1 align="center"><strong>Does size matter?</strong></h1>
  <p align="center">
    Applied Data Analysis (CS-401)
  </p>
</div>

Customers are increasingly relying on **product rating** websites to inform their purchasing decisions. It has been demonstrated that when customers rate a product, they often exhibit a **tendency to be influenced by the previous ratings** of other customers, a phenomenon known as the **_herding effect_**.

Despite this, an unresolved research question revolves around comprehending **how ratings might be impacted by the scale and the reputation of the vendor**. Utilizing data sourced from beer reviews websites, our objective is to investigate the **connection** between the **size and fame of vendors** (specifically, breweries) and **the perceived quality** of their products.

Through the quantification of brewery size and popularity using **predefined metrics** and the **extraction of sentiment** from textual reviews, our aim is to ascertain whether a correlation exists between vendor size and notoriety and perceived product quality. Additionally, we plan to **explore the behaviors** of diverse consumer bases, considering **temporal dimensions** (how these phenomena have evolved over the years and seasons within the same year) and **spatial dimensions** (how these relationships differ across states and countries).


## **BeerAdvocate**: Final Analysis

In [ ]:
# Import  libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
import os
from datetime import datetime
import help_functions as helpfn

In [ ]:
# Define the data folder paths for BeerAdvocate
DATA_FOLDER_BA = 'DATA/BeerAdvocate/'

# Define the file paths for the datasets
Beers_DATASET = DATA_FOLDER_BA+"beers.csv"
Users_DATASET = DATA_FOLDER_BA+"users.csv"
Reviews_DATASET = DATA_FOLDER_BA+"reviews_BA.csv"
Breweries_DATASET = DATA_FOLDER_BA+"breweries.csv"

# 1. Dataset Exploration & Cleaning

The first step of the project consits in exploring the data available in the dataset as well as pre-processing them. This step consists mainly in handling the potential missing data and reformatting what needs to be reformated. The following subsections present this step for every datasets available.

## 1.1 Beers

We starts with the dataset containing information about beers. We will probably not use this datatset since our project focuses more on the reviews and breweries but we decided to still perform some pre-processing on it in the case where we suddenly need it during the analysis of Milestone 3. The dataset includes the following columns:

- `beer_id`: Identifier for the beer.
- `beer_name`: Name of the beer.
- `brewery_id`: Identifier for the brewery producing the beer.
- `brewery_name`: Name of the brewery.
- `style`: Beer style.
- `nbr_ratings`: Number of ratings received.
- `nbr_reviews`: Number of reviews.
- `avg`: Average rating.
- `ba_score`: BeerAdvocate score.
- `bros_score`: Bros score.
- `abv`: Alcohol by volume.
- `avg_computed`: Computed average rating.
- `zscore`: Z-score.
- `nbr_matched_valid_ratings`: Number of matched valid ratings.
- `avg_matched_valid_ratings`: Average of matched valid ratings.


In [ ]:
# Read the datasets into Pandas DataFrames
beers = pd.read_csv(Beers_DATASET)
# Display 2 random chosen samples of the set
display(beers)

We can see that the dataset is composed of 280'823 beers. However, we can see that some data are missing and that some beers have 0 reviews. Furthermore, we should check if there is some duplicates. To address these issues, the following pre-processing steps are applied:

- **Filtering Beers with Less than 5 Reviews**: Deleting beers with fewer than 5 reviews, as they may not be characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_ratings` column.

- **Removing Duplicates**: Dropping duplicate entries based on the `beer_name` and `beer_id` columns.

- **Column Selection**: Dropping columns that won't be used in our analysis.

In [ ]:
# Set a minimum threshold for the number of reviews
MIN_NUMBER_OF_REVIEWS = 5

# Create a filtered copy of the 'beers' DataFrame with a minimum number of reviews
beers_filt = beers.copy(deep=True)
beers_filt = beers_filt[beers_filt['nbr_reviews'] >= MIN_NUMBER_OF_REVIEWS]

# Remove rows with missing values in the 'nbr_reviews' column
beers_filt = beers_filt[beers_filt['nbr_reviews'].notna()]

# Drop duplicate entries based on the 'beer_name' column
beers_filt = beers_filt.drop_duplicates(subset=['beer_name'])

# Calculate the number of duplicate entries based on 'beer_name' and 'beer_id'
dupli_name = np.sum(beers_filt.duplicated(subset=['beer_name']))
dupli_ID = np.sum(beers_filt.duplicated(subset=['beer_id']))

# Drop specific columns from the filtered DataFrame
beers_filt = beers_filt.drop(['zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings', 'bros_score', 'ba_score'], axis=1)

# Display the filtered DataFrame
display(beers_filt)

# Print the number of duplicate entries for 'beer_name' and 'beer_id'
print(f'Number of duplicate beer name = {dupli_name}')
print(f'Number of duplicate beer ID = {dupli_ID}')


After pre-processing, the number of beers drops to 42'923.

## 1.2 Users

We then move to the pre-processing of the dataset containing information about users. The dataset includes the following columns:

- `nbr_ratings`: Number of ratings made.
- `nbr_reviews`: Number of reviews done.
- `user_id`: Unique user identifier.
- `user_name`: User Name.
- `joined`:  Date of the sign up.
- `location`: Location of the User.


In [ ]:
# Read the datasets into Pandas DataFrames
users = pd.read_csv(Users_DATASET)
display(users)

The dataset is composed of 153'704 users. We can again see that some data are missing and that some users did not gave any review. To address these issues our pre-processing involves the following steps:

- **Filtering Users with 0 number of reviews**: Deleting users with 0 reviews, as they are not characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_reviews`, `user_id`, `user_name` and `location` columns.

- **Check for Duplicated Users**: Check if there are multiple user with the same id.

- **Formatting the date**: Reformat the date in the column `joined` in UTC format.

In [ ]:
# Create a deep copy of the 'users' dataframe
users_filt = users.copy(deep=True)

# Check for duplicates based on user name and user ID
dupli_name = np.sum(users_filt.duplicated(subset=['user_name']))
dupli_ID = np.sum(users_filt.duplicated(subset=['user_id']))

# Remove users with 0 reviews and NaN as the number of reviews
users_filt = users_filt[users_filt['nbr_reviews'] >= 1]
users_filt = users_filt[users_filt['nbr_reviews'].notna()]

# Remove rows with NaN in 'user_id', 'user_name', and 'location'
users_filt = users_filt[users_filt['user_id'].notna()]
users_filt = users_filt[users_filt['user_name'].notna()]
users_filt = users_filt[users_filt['location'].notna()]

# Convert 'joined' column to datetime type
users_filt['joined'] = users_filt['joined'].apply(lambda x: datetime.utcfromtimestamp(x) if not pd.isna(x) else x)

# Rename the 'location' column to 'user_location'
users_filt.rename(columns={'location': 'user_location'}, inplace=True)

# Display the resulting dataframe
display(users_filt)

# Display the number of duplicate user names and user IDs
print(f'Number of duplicate user names = {dupli_name}')
print(f'Number of duplicate user IDs = {dupli_ID}')

# Display the number of NaN values in each category
print('Number of NaN by category:')
print(np.sum(users_filt.isna()))


After pre-processing, a total of 58'199 users with complete information are kept.

## 1.3 Breweries

Next, we conduct the pre-processing of the dataset containing information about breweries. The dataset comprises the following columns:

- `id`: Brewery identifier.
- `location`: Location of the brewery.
- `name`: Name of the brewery.
- `nbr_beers`: Number of beers produced.


In [ ]:
# Read the datasets into Pandas DataFrames
breweries = pd.read_csv(Breweries_DATASET)
display(breweries)

The dataset is composed of 16'758 breweries. We can directly see that some breweries have 0 beers and are therefore not relevant for our analysis. There might be some missing values as well, thus we decided to perform the following pre_processing steps:

- **Filtering Breweries with 0 number of beers**: Deleting breweries with 0 beers, as they are not characteristic.

- **Handling Missing Values**: Dropping rows with NaN values in the `nbr_beers`.

- **Removing Duplicates**: Dropping duplicate entries based on the `name` column.

In [ ]:
# Create a deep copy of the 'breweries' dataframe
breweries_filt = breweries.copy(deep=True)

# Check for duplicates based on ID
dupli_ID = np.sum(breweries_filt.duplicated(subset=['id']))

# Remove breweries with 0 beers and NaN values
breweries_filt = breweries_filt[breweries_filt['nbr_beers'] >= 1]
breweries_filt = breweries_filt[breweries_filt.notna()]

# Remove duplicate entries based on brewery name
breweries_filt = breweries_filt.drop_duplicates(subset='name')

# Check for duplicates based on name
dupli_name = np.sum(breweries_filt.duplicated(subset=['name']))

# Rename columns for consistency
breweries_filt.rename(columns={'name': 'brewery_name', 'id': 'brewery_id', 'location': 'brewery_location'}, inplace=True)

# Display the resulting dataframe
display(breweries_filt)

# Display the number of duplicate names and IDs
print(f'Number of duplicate names = {dupli_name}')
print(f'Number of duplicate IDs = {dupli_ID}')

# Display the number of NaN values in each category
print('Number of NaN by category:')
print(np.sum(breweries_filt.isna()))


After pre-processing, 14'158 breweries are remaining. 

## 1.4 Reviews

We now delve into the BeerAdvocate Reviews dataset, focusing on reviews of various beers. The dataset contains the following columns:

- `beer_name`: Name of the beer.
- `beer_id`: Identifier for the beer.
- `brewery_name`: Name of the brewery producing the beer.
- `brewery_id`: Identifier for the brewery.
- `style`: Beer style.
- `abv`: Alcohol by volume.
- `date`: Timestamp of the review.
- `user_name`: Username of the reviewer.
- `user_id`: Identifier for the user.
- `appearance`: Rating for the beer's appearance.
- `aroma`: Rating for the beer's aroma.
- `palate`: Rating for the beer's palate.
- `taste`: Rating for the beer's taste.
- `overall`: Overall rating.
- `rating`: Overall user rating.
- `text`: Review text.

Since the .txt file containing the reviews is huge, we first converted it into smaller files that we then regrouped in a .csv file. The code in `split_reviews.py` was used or this purpose.

In [ ]:
# Read the Reviews dataset into a pandas DataFrame.
reviews_BA = pd.read_csv(Reviews_DATASET)
display(reviews_BA)

Our pre-processing begins with the following initial steps:

- **Converting Timestamps to Datetime**: We start by converting the 'date' column, which contains timestamps, into the datetime format. This conversion enables us to perform time-based analyses more effectively.

- **Handling Missing Values**: We address missing values in the dataset by dropping rows with NaN values. This ensures that our analysis is based on complete and reliable data.

- **Removing Unnecessary Column**: The 'abv' column, representing the alcohol by volume, is not useful for our specific analysis. Consequently, we opt to drop this column to streamline our dataset.

- **Removing White Space before and after Stings**: Some strings columns like `user_name` begin with a white space, which is a problem for merging. Consequently we need to remove them.

In [ ]:
# Create a deep copy of the reviews_BA DataFrame to avoid modifying the original DataFrame.
reviews_filt = reviews_BA.copy(deep=True)
# Convert the 'date' column to a datetime format.
# If the 'date' value is not NaN, apply the conversion using utcfromtimestamp.
# If the 'date' value is NaN, leave it unchanged.
reviews_filt['date'] = reviews_filt['date'].apply(lambda x: datetime.utcfromtimestamp(x) if not pd.isna(x) else x)
# Drop rows where the 'text' column has NaN values.
reviews_filt = reviews_filt[reviews_filt['text'].notna()]
# Drop the 'abv' column from the reviews_filt DataFrame.
reviews_filt = reviews_filt.drop(['abv'], axis=1)
# Remove leading and trailing whitespaces (if they exist) from the following columns: 
# user_id, user_name, beer_name, brewery_name, style, and text.
reviews_filt.user_id = reviews_filt.user_id.apply(lambda x: x.strip())
reviews_filt.user_name = reviews_filt.user_name.astype(str).apply(lambda x: x.strip())
reviews_filt.beer_name = reviews_filt.beer_name.apply(lambda x: x.strip())
reviews_filt.brewery_name = reviews_filt.brewery_name.apply(lambda x: x.strip())
reviews_filt['style'] = reviews_filt['style'].apply(lambda x: x.strip())
reviews_filt.text = reviews_filt.text.apply(lambda x: x.strip())
# Display the updated reviews_filt DataFrame.
display(reviews_filt)
# Print the number of NaN values for each column in the reviews_filt DataFrame.
print('Number of NaN by category:')
print(np.sum(reviews_filt.isna()))


We now aim to visualize the distribution of the length of the reviews to:

- **Get insight into review length variation**: Visualizing the distribution allows us to understand the range and variability in review lengths. Some reviews may be succinct, while others may be more detailed.

- **Assess data quality**: Analyzing review lengths can also serve as a quality check. Unusually short or long reviews may warrant further investigation to ensure data integrity.


In [ ]:
# Create a histogram of the review lengths using the 'text' column from the reviews_filt DataFrame.
plt.hist(reviews_filt['text'].str.len(), bins=200, log=True)

# Set x-axis and y-axis labels and the title.
plt.xlabel('Review length')
plt.ylabel('Number of reviews')
plt.title('Distribution of Review Length')

plt.show()


The BeerAdvocate website advises to create reviews of at list 150 characters. It can be seen that not all the reviews have at least 150 characters. We will therefore remove them in the next steps to keep only relevant revies. The statistics of the review length are displayed in the next cell. As we can see, the median is at 580 characters. The distribution is skewed though, with a small quantity of reviews being more than 5000 characters. This will be interesting to analyse in the next milestone if there is a link between the length of the reviews and the scale or popularity of a brewery.

In [ ]:
# Display descriptive statistics of the review lengths.
reviews_filt['text'].str.len().describe()

We attempted to improve consistency by **translating** all non-English textual reviews. To this end, we used the language detection module, $\texttt{detect}$, of the $\texttt{langdetect}$ library to **initially identify the language of each review**.

Due to the considerable computation time required for language detection, we decided to **keep the language identifier** of each review in a separate dataset, together with the **unique identifiers** of the **beer** and the **user**.

This approach allows us to store the language information in our archive, facilitating efficient access without the need to calculate the language detection for each review each time.

Note that if the CSV file containing `user_id`, `beer_id` and `text_lang` (the language identifier of the reviews) exists in the repository, we can avoid recomputing the information. Instead, we can merge the review dataset with this auxiliary dataset into a consolidated dataset, simplifying our analysis process.

In [ ]:
# The next sections will need these modules to be run
# Importing the 'unescape' function from the 'html' module for text cleaning of html escape characters
try:
    from html import unescape
except:
    !pip install html

# Importing the 'detect' function from the 'langdetect' module for language detection of reviews
try:
    from langdetect import detect
except:
    !pip install langdetect

# Importing the 'GoogleTranslator' from the 'deep_translatore' module for reviews translation
try:
    from deep_translator import GoogleTranslator
except:
    !pip install deep-translator

In [ ]:
# If the .csv file exists then we don't redo the detection
data_name = 'reviews_lang.csv'
CODE_ERROR = 'Error'

if os.path.exists(data_name):
    reviews_language = pd.read_csv(data_name)
    reviews_filt = pd.merge(reviews_filt, reviews_language, on=['beer_id','user_id'], how='left')
else:
    # Detect the language of each review. Handle exception for non corrected reviews.
    text_lang = []
    for review in reviews_filt['text']:
        try:
            text_lang.append(detect(review))
        except:
            text_lang.append(CODE_ERROR)
            continue
    
    # Adding a new column 'text_lang' to store the detected language for each review
    reviews_filt['text_lang'] = pd.Series(text_lang)

    # Store the language information in a  
    reviews_filt[['beer_id', 'user_id', 'text_lang']].to_csv(data_name, index=False)

We can look at the consolidated dataset with the new column giving the language of the review.

In [ ]:
# display 3 randomly chosen rows of the new consolidated dataset
display(reviews_filt.sample(3))

Now we can check if errors occured during the detection, as well as if NaN values appeared in the process.

In [ ]:
print('The number of errors detected are:', np.sum(reviews_filt['text_lang'] == CODE_ERROR))

In [ ]:
print(f'The number of NaN values in text_lang colum is:', np.sum(reviews_filt['text_lang'].isna()), '/', len(reviews_filt['text_lang']))

We then decide to drop the NaN values due to the small number of occurrences of NaN values in the `text_lang` column in our filtered reviews dataset.

In [ ]:
reviews_filt = reviews_filt[reviews_filt['text_lang'].notna()]

We can now have a look to the variety of languages composing the reviews. As expected a vast majority of them are in English.

In [ ]:
print('The count and variety of distinct languages used in the reviews within our dataset:')
print(reviews_filt['text_lang'].value_counts())

We then filter rows in the DataFrame where the `text_lang` column is not 'en', and then apply translation to English for the corresponding 'text' column.

In [ ]:
reviews_filt[reviews_filt.text_lang != 'en']['text'] = reviews_filt[reviews_filt.text_lang != 'en']['text'].apply(lambda x: helpfn.translate_to_english(x))

With all reviews now in **English**, we can proceed to remove the `text_lang` column.

In [ ]:
reviews_filt = reviews_filt.drop(['text_lang'], axis=1)

We then undertook further pre-processing, focusing on the textual representation of reviews. We followed the following steps:

- **Management of special characters**: After examining the modified and translated dataset, we observed the presence of some special characters such as "\&quot;" and "\x92" in some reviews. To solve this problem, we used the html.unescape function to convert the HTML entities and then removed the non-ASCII characters by encoding them in ASCII and decoding them again.

- **Filtering short reviews**: As a final step, we filtered out reviews with less than 150 characters. This step aimed to exclude shorter reviews from our dataset, focusing on more substantial texts.

In [ ]:
# Apply the 'unescape' function to decode HTML entities in the 'text' column
reviews_filt['text'] = reviews_filt['text'].apply(unescape)

# Remove non-ASCII characters by encoding to ASCII and decoding back
reviews_filt['text'] = reviews_filt['text'].apply(lambda x: x.encode('ascii', 'ignore').decode())

# Set the minimum number of characters for reviews
MIN_NUMBER_OF_CHARACTER = 150

# Filter out reviews with fewer than 'min_character' characters
reviews_filt = reviews_filt[reviews_filt['text'].str.len() > MIN_NUMBER_OF_CHARACTER]

In [ ]:
display(reviews_filt)

We end up with 2'587'598 reviews. Now that the datasets are ready, we can start with some preliminary analysis.

# 2 Preliminary Analysis
The website offers data since 1996. However we should consider that at the very begining of the website only ridiculously small portion of people had access to internet. Therefore we do some analysis of the evolution of the number of revoruenavui  blabla bla

Plot of nbr revies and breweries evolution through years.

--> conclude that we only keep things from ?? 2000 ??

## 2.1 Temporal Evolution of # breweries and # reviews

In [ ]:
reviews_filt['date']= reviews_filt['date'].apply(lambda x: x.year)
temp_df = reviews_filt[['brewery_id','date']].copy(deep=True)
reviews_by_year = temp_df.groupby('date').size().reset_index(name='tot_nbr_reviews_by_year')
brew_by_year = temp_df.groupby('date')['brewery_id'].unique().reset_index(name='brewery')
brew_by_year['tot_nbr_brew_by_year'] = brew_by_year['brewery'].apply(lambda x: len(x))
temporal_df = pd.merge(reviews_by_year,brew_by_year, on=['date'],how='inner')
temporal_df['rev_by_brew'] = temporal_df.apply(lambda row: row.tot_nbr_reviews_by_year/row.tot_nbr_brew_by_year, axis = 1)


In [ ]:
# Sort the DataFrame by 'date' if it's not sorted already
df = temporal_df.sort_values(by='date')

# Define pastel colors
pastel_blue = '#AED6F1'
pastel_red = '#F1948A'
pastel_green = '#ABEBC6'

# Create subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12))

# Bar plot for total number of reviews by year
bars1 = ax1.bar(df['date'], df['tot_nbr_reviews_by_year'], color=pastel_blue, alpha=0.7, edgecolor='black')
ax1.set_title('Total Number of Reviews by Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Number of Reviews')
for bar in bars1:
    yval = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Bar plot for total number of breweries by year
bars2 = ax2.bar(df['date'], df['tot_nbr_brew_by_year'], color=pastel_red, alpha=0.7, edgecolor='black')
ax2.set_title('Total Number of Breweries Reviewed by Year')
ax2.set_xlabel('Year')
ax2.set_ylabel('Total Number of Breweries')
for bar in bars2:
    yval = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Bar plot for ratio
bars3 = ax3.bar(df['date'], df['rev_by_brew'], color=pastel_green, alpha=0.7, edgecolor='black')
ax3.set_title('Ratio between number of reviews and number of breweries by year')
ax3.set_xlabel('Year')
ax3.set_ylabel('#reviews/#breweries')
for bar in bars3:
    yval = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')


# Adjust layout for better appearance
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
reviews_filt = reviews_filt[reviews_filt['date']>=2002]

## 2.2 Metrics Definition
In our analysis, we introduce **two key metrics** to quantitatively **assess the characteristics of breweries**: Size Metrics and Popularity Metrics. These metrics are created in the next cells. Since we don't have data about the revenue or number of liter produced by the brewery, we decided to base ourself on variable present in the dataset to construct these metrics. Some verification will be done by hand, by searching information about some breweries on the web to assess the quality of the metrics and to correct the coefficients if needed.

####  **Size Metrics**
To numerically evaluate the size of a brewery over the years, we built an index based on the following formula:

$$ \text{Size} = \alpha \log (N_r) + \beta \log (N_b) + \gamma \log (N_t)$$

With:
- $N_r =$ number of reviews received
- $N_b =$ number of beers produced
- $N_t =$ number of different types (style) of beer produced

The coefficients $\alpha, \beta, \gamma$ are chosen by hand. The metrics is computed for each brewery every year. For each year the value obtained for each brewery is normalized by the maximum value of the year, obtaining thus a value between 0 and 1 for each brewery, with the biggest brewery having a size index of 1.

#### **Popularity Metrics**
$$ \text{Popularity} = \dfrac{\log (N_r)}{N_b}$$

With $N_r =$ number of reviews, $N_b =$ number of beers produced


Note that both metrics are normalized to get a value between 0 and 1.

In [ ]:
# Extract brewery_id, style, beer_id and date  columns from reviews_filt
review_brew = reviews_filt[['brewery_id', 'style', 'beer_id', 'date', 'rating']].copy(deep=True)

reviews_filt['text'].str.len()

review_brew['len_text'] = reviews_filt['text'].apply(lambda x: len(x))

# Group by brewery ID and Year of review and compute nbr of reviews by year
brew_review_score = review_brew.groupby(['brewery_id','date']).size().reset_index(name='nbr_reviews')

# Count the number of unique styles for each brewery by year
brew_style_score = review_brew.groupby(['brewery_id','date'])['style'].unique().reset_index(name='nbr_styles')
brew_style_score['nbr_styles'] = brew_style_score['nbr_styles'].apply(lambda x: len(x))

# Count the number of unique beer reviewed for each brewery by year
brew_beer_score = review_brew.groupby(['brewery_id','date'])['beer_id'].unique().reset_index(name='nbr_beers')
brew_beer_score['nbr_beers'] = brew_beer_score['nbr_beers'].apply(lambda x: len(x))

# Compute average ratings for each brewery by year
brew_rating_score = review_brew.groupby(['brewery_id','date'])['rating'].mean().reset_index(name='avg_rating')

# Compute the mean review length for each brewery by year
brew_mean_len = review_brew.groupby(['brewery_id','date'])['len_text'].mean().reset_index(name='mean_len_text')

# Merge the review count, style count, and selected columns from breweries_filt
brew_by_year = pd.merge(brew_review_score, brew_style_score[['brewery_id', 'date', 'nbr_styles']], left_on=['brewery_id','date'] , right_on=['brewery_id','date'], how='inner')
brew_by_year = pd.merge(brew_by_year, brew_beer_score[['brewery_id', 'date', 'nbr_beers']], left_on=['brewery_id','date'] , right_on=['brewery_id','date'], how='inner')
brew_by_year = pd.merge(brew_by_year, brew_rating_score[['brewery_id', 'date', 'avg_rating']], left_on=['brewery_id','date'] , right_on=['brewery_id','date'], how='inner')
brew_by_year = pd.merge(brew_by_year, brew_mean_len[['brewery_id', 'date', 'mean_len_text']], left_on=['brewery_id','date'] , right_on=['brewery_id','date'], how='inner')
brew_by_year = pd.merge(brew_by_year, reviews_by_year, on=['date'], how='inner')

# Drop if only 1 review
brew_by_year = brew_by_year[brew_by_year['nbr_reviews']>=2]
display(brew_by_year)

In [ ]:

df_metrics = brew_by_year.copy(deep=True)
# /row.tot_nbr_reviews_by_year ??
df_metrics['size_metrics'] = df_metrics.apply(lambda row: 5*np.log(row.nbr_reviews) + np.log(row.nbr_styles) + 2*np.log(row.nbr_beers), axis = 1)
# Apply the normalization within each year
df_metrics['size_metrics'] = df_metrics.groupby('date')['size_metrics'].transform(helpfn.normalize_column)

df_metrics['popularity_metrics'] = df_metrics.apply(lambda row: np.log(row.nbr_reviews)/row.nbr_beers, axis = 1)
df_metrics['popularity_metrics'] = df_metrics.groupby('date')['popularity_metrics'].transform(helpfn.normalize_column)
display(df_metrics)


In [ ]:
size_metric_macro = df_metrics.groupby('brewery_id')['size_metrics'].mean().reset_index(name='size_metrics_macro')
popularity_metric_macro = df_metrics.groupby('brewery_id')['popularity_metrics'].mean().reset_index(name='popularity_metrics_macro')
avg_rating_macro = df_metrics.groupby('brewery_id')['avg_rating'].mean().reset_index(name='avg_rating_macro')
avg_len_text_macro = df_metrics.groupby('brewery_id')['mean_len_text'].mean().reset_index(name='avg_len_text_macro')
df_metrics_macro = pd.merge(size_metric_macro,popularity_metric_macro, on=['brewery_id'], how='inner')
df_metrics_macro = pd.merge(df_metrics_macro,avg_rating_macro, on=['brewery_id'], how='inner')
df_metrics_macro = pd.merge(df_metrics_macro,avg_len_text_macro, on=['brewery_id'], how='inner')
df_metrics_macro = pd.merge(df_metrics_macro,breweries_filt[['brewery_id','brewery_name']],on=['brewery_id'], how='inner')


display(df_metrics_macro)

In [ ]:
# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot histogram for 'size metrics' in the first subplot
axs[0].hist(df_metrics_macro['size_metrics_macro'], bins=20,color=pastel_blue, edgecolor='black')
axs[0].set_title('Distribution of the breweries based on size metrics')
axs[0].set_xlabel('Size metrics')
axs[0].set_ylabel('Number of breweries')

# Plot histogram for 'popularity metrics' in the second subplot
axs[1].hist(df_metrics_macro['popularity_metrics_macro'], bins=20, color=pastel_green, edgecolor='black')
axs[1].set_title('Distribution of the breweries based on popularity metrics')
axs[1].set_xlabel('Popularity metrics')
axs[1].set_ylabel('Number of breweries')

# Set y-axis scale to logarithmic for better visibility of distribution in both subplots
#axs[0].set_yscale('log')
#axs[1].set_yscale('log')

# Adjust layout
plt.tight_layout()

plt.show()

The Popularity metrics.................... But let's see if the biggest breweries are also the most popular ones.

In [ ]:
# Sort breweries by size
size_sorted = df_metrics_macro.sort_values(by='size_metrics_macro', ascending=False).reset_index()
size_sorted.index = np.linspace(1, len(size_sorted), len(size_sorted)).astype(int)
print("Overview of the size metrics ranking:")

# Display a summary of the top breweries based on size metrics
display(size_sorted[['brewery_name', 'size_metrics_macro']].iloc[[0, 1, 2, 9, 99, 999]])

When sorting the breweries by size, it becomes evident that the top-ranking breweries are primarily located in the US. This observation aligns with expectations, considering that the majority of reviewers and breweries are located in the US (as detailed in the following section).

Looking for the 3 biggest and the 100th on the web, we can extract their beer production:

- (1st) Boston Beer Company: 5,300,000 barrels (6,200,000 hL) found under https://en.wikipedia.org/wiki/Boston_Beer_Company
- (2nd) Sierra Nevada Brewing Co.: 1,250,000 barrels (510,000 hL) found under https://en.wikipedia.org/wiki/Sierra_Nevada_Brewing_Company
- (3rd) Stone Brewing: 325,645 barrels (382,000 hL) found under https://en.wikipedia.org/wiki/Stone_Brewing_Co.
- (100th) Uinta Brewing Company: 77'000 barrels found under https://en.wikipedia.org/wiki/Uinta_Brewing_Company

Given these values, it seems that the size metrics works, at least for the top ranked breweries.

On the other side, the popularity metrics yield significantly different results. The top-ranked breweries include historically renowned Belgian breweries such as "Brasserie de Rochefort," notable for its "Trappist" beer, and "Brasserie d'Orval." This highlights that the popularity metric captures a different aspect of brewery influence, focusing on factors beyond sheer size.

It is important to note that this preliminary analysis does not take into account the time evolution. It will be interesting to see how it has evolved over the years. Some breweries for example could have significantly changed size over the years. However this pre-analysis validates the choice of the metrics and opens a large range of possibilities for the next analysis.


### 2.2.1 Evolution of the Size Metrics over the Year

Let's take a look at how brewery size distribution has changed over the years. 

In [ ]:
mean_size_year = df_metrics.groupby('date')['size_metrics'].mean()
std_size_year = df_metrics.groupby('date')['size_metrics'].std()

In [ ]:
# Histogram of the brewery size metrics definition
years = df_metrics['date'].drop_duplicates()
plt.figure(figsize=(8, 6))
h,_,_,sc=plt.hist2d(df_metrics.date, df_metrics.size_metrics, norm=LogNorm(), bins=[np.linspace(2001.5, 2017.5, 17),25], cmap='Reds')
plt.errorbar(years, mean_size_year, yerr=std_size_year, fmt='o-', color='white', ecolor='white',label='Yearly mean with std of size metrics', capsize=5, alpha=1.0)

plt.colorbar(sc)
plt.ylabel('Size metrics')
plt.xlabel('years')
plt.legend()
plt.title('Evolution of the size metrics distribution according to time [years]')
plt.show()

The distribution of brewery sizes has remained fairly stable over the years. Nevertheless, since around 2014, the proportion of small breweries has tended to increase substantially. This growth can be partly explained by the strong interest in breweries and the creation of craft beers observed between 2015 and 2020. Google searches for the term brewery, for example, experienced strong increases during this period, which matches the results given by our size metric.

![Brewery Google query time evolution](images/google_search_brewery.png)

### 2.2.2 Small, medium, big breweries
Grouping by small, medium and big breweries according to the size metric

In [ ]:
df_metrics_macro['size_category'] = df_metrics_macro['size_metrics_macro'].apply(helpfn.categorize_size)
df_metrics['size_category'] = df_metrics['size_metrics'].apply(helpfn.categorize_size)

display(df_metrics_macro)


Let's see how the breweries are divide into the size categories 

In [ ]:
# Compute the percentage of breweries by size categories
perc_small_breweries = (df_metrics_macro['size_category']=='small').sum() / df_metrics_macro.shape[0] * 100
perc_medium_breweries = (df_metrics_macro['size_category']=='medium').sum() / df_metrics_macro.shape[0] * 100
perc_big_breweries = (df_metrics_macro['size_category']=='big').sum() / df_metrics_macro.shape[0] *100

print(f"Percentage of small breweries: {perc_small_breweries:.2f}",'%')
print(f"Percentage of medium breweries: {perc_medium_breweries:.2f}",'%')
print(f"Percentage of big breweries: {perc_big_breweries:.2f}",'%')

As expected, the vast majority of breweries are small and only a minority are big breweries, probably with an international reach.

## 2.3 Geographical Data

The goal of this section is to gain insights into the geographical distribution of both breweries and reviewers within the dataset. Ultimately, we aim to calculate the distances between breweries and their respective reviewers. This analysis could potentially unveil distinctions between brewery types, revealing whether certain types of breweries attract predominantly local reviewers or have a more globally dispersed audience.

### 2.3.1 Spatial Distribution of Breweries

We initiate our analysis by examining the geographical distribution of breweries in the dataset. To achieve this, we integrate the dataset with a map sourced from Geopandas (https://www.naturalearthdata.com/downloads/110m-cultural-vectors/).

Ensuring alignment between the country names used in the map and those in the brewery dataset is crucial. To address this, we calculate the Hamming distance between them and substitute the brewery location with the closest match. In instances where no match is found, we opt to eliminate the corresponding brewery. This scenario applies to 26 breweries out of over 14,000, which is deemed acceptable for this level of analysis. It's worth noting that some removed breweries had HTTP links as their location, explaining the lack of match with the map.

For breweries located in the USA, the dataset includes information about the state. Consequently, we extend the same process to the states in the United States of America. To facilitate this, we introduce a new column, 'state,' in the dataframe. This allows us to split the location of US breweries into 'country' and 'state.' 

The geodataframe for the USA can be accessed here: https://eric.clst.org/tech/usgeojson/

In [ ]:
# Make deep copy of breweries filtered dataframe reset indexes (needed for the loop next)
breweries_loc = breweries_filt.copy(deep=True)
breweries_loc.reset_index(inplace=True,drop=True)

# Load the world map shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Load the USA state map
us_states = gpd.read_file('GeoJSON/gz_2010_us_040_00_500k.json')
# Change name of Georgia into Georgia USA (sae for Puerto Rico)
us_states.loc[28,'NAME'] = 'Georgia USA'
us_states.loc[16,'NAME'] = 'Puerto Rico USA'
# Extract countries name
unique_loc = world['name'].unique()
# Extract state name
unique_state = us_states['NAME'].unique()

# Create column for state in breweries df, will be used for USA
breweries_loc['state'] = '-'
breweries_loc.rename(columns={'brewery_location': 'country'}, inplace=True)

breweries_loc, dropped_brew = helpfn.match_country_name(breweries_loc,
                                                 unique_loc,
                                                 unique_state)

# Display a sample of the modified dataframe
breweries_loc.rename(columns={'country':'brewery_country','state':'brewery_state'}, inplace=True)
display(breweries_loc.sample(3))

# Print the number of dropped breweries and their locations
print(f'{len(dropped_brew)} breweries were dropped, at the following location:')
print(dropped_brew)


We explore the **Top 10 countries** with the **highest number of breweries** to gain insights into the **global distribution of brewing establishments**.

In [ ]:
# Count the number of breweries in each country
brewery_counts = breweries_loc['brewery_country'].value_counts().reset_index()
brewery_counts.columns = ['brewery_country', 'nb_breweries']

# Identify the top 10 countries
top10 = brewery_counts.sort_values(by='nb_breweries', ascending=False).head(10)
top10 = top10[['brewery_country', 'nb_breweries']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_breweries = top10.nb_breweries.astype(int)

# Display the Top 10 countries with the most breweries
print('Top 10 countries with the most breweries:')
display(top10)

As evident from the data, **there are significantly more breweries in the US** compared to other countries. Due to this notable concentration, we will consider the breweries by states in the next section. To facilitate this, we extend the world geodataframe by incorporating the US state geodataframe in the next section.

In [ ]:
# Add US states to world df
us_states.rename(columns={'NAME': 'name'}, inplace = True)
world_with_US_states = pd.concat([world[['name','geometry']],us_states[['name','geometry']]])
duplicated_rows = world_with_US_states['name'].duplicated(keep='first')

# Drop rows with duplicate values in Column1
world_with_US_states = world_with_US_states[~duplicated_rows]

Let's look again to the Top 10 countries with the highest number of breweries. However, this time, we'll consider **each US state as a distinct 'country'.** Additionally, we'll **visualize the global distribution of breweries** to gain a comprehensive understanding of their geographical spread.

In [ ]:
# Count the number of breweries in each country
brewery_counts = breweries_loc['brewery_state'].value_counts().reset_index()
brewery_counts.columns = ['brewery_state', 'nb_breweries']

# Merge brewery counts with the world map data
world_merge = world_with_US_states.merge(brewery_counts, how='left', left_on='name', right_on='brewery_state')

# Fill NaN values (countries without breweries) with 0
world_merge['nb_breweries'].fillna(0, inplace=True)
# Fill NaN values of state with Unknown
world_merge['brewery_state'].fillna('Unknown', inplace=True)

# Find top 10 countries
top10 = world_merge.sort_values(by='nb_breweries', ascending=False).head(10)
top10 = top10[['name', 'nb_breweries']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_breweries = top10.nb_breweries.astype(int)

# Display the Top 10 countries (considering US states) with the most breweries
print('Top 10 countries (considering US states) with the most breweries:')
display(top10)

# Plot the choropleth map
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
world_merge.boundary.plot(ax=ax, color='black', linewidth=0.5)
world_merge.plot(column='nb_breweries', ax=ax, legend=True,
                 norm=LogNorm(vmin=1, vmax=world_merge['nb_breweries'].max()),
                 legend_kwds={'label': "Number of Breweries by Country",
                              'orientation': "vertical"},
                 cmap='Reds')

# Remove the axis
ax.set_title('Distribution of the breweries in the World')
ax.set_axis_off()
plt.show()


The revised analysis reveals distinct results. Germany takes the lead, followed by Finland, and several US states are now present in the Top 10. The conclusion of this analysis is that there are many more breweries in the US than anywhere else in the world. Acknowledging this, it becomes essential to consider the US's internal diversity in subsequent analyses. Given the availability of information about individual US states, incorporating them into the analysis could provide valuable insights, offering a more nuanced understanding of the brewing landscape within the country.

### 2.3.2 Spatial Distribution of the Users
Now that we have gained insights into the distribution of breweries worldwide, we can apply a **similar analysis** to the **user data**. The aim of this analysis is to see whether or not a geographical similarity can be observed between the distribution of breweries and the distribution of users who give reviews. Once again, a matching process based on Hamming's distance must be performed to match the names of the different places.

In [ ]:
# Create a copy of the filtered users dataset and reset indexes (needed for the loop next)
users_loc = users_filt.copy(deep=True)
users_loc.reset_index(inplace=True,drop=True)
# Create a column for the state in the users dataframe, to be used for USA
users_loc['state'] = '-'
users_loc.rename(columns={'user_location': 'country'}, inplace=True)
# Init dropped breweries
#dropped_brew = []
users_loc, dropped_user = helpfn.match_country_name(users_loc,
                                             unique_loc,
                                             unique_state)

# Display a sample of the modified dataframe
users_loc.rename(columns={'country':'user_country','state':'user_state'}, inplace=True)

# Display a sample of the modified users dataframe
display(users_loc.sample(3))

# Print the number of dropped users and their locations
print(f'{len(dropped_user)} users were dropped, at the following location:')
print(dropped_user)

Let's examine the **Top 10 countries** with the **highest number of reviewers** to gain insights into the global distribution of reviewers.

In [ ]:
# Count the number of reviewers in each country
reviewers_counts = users_loc['user_country'].value_counts().reset_index()
reviewers_counts.columns = ['user_country', 'nb_reviewers']

# Identify the top 10 countries according to the number of reviewers
top10 = reviewers_counts.sort_values(by='nb_reviewers', ascending=False).head(10)
top10 = top10[['user_country', 'nb_reviewers']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_reviewers = top10.nb_reviewers.astype(int)

# Display the Top 10 countries with the most reviewers
print('Top 10 countries with the most reviewers:')
display(top10)

As observed, there is once again a **substantial number of reviewers in the US**. The distribution seems fairly similar to that observed for breweries. In the next section, we will **replicate the analysis**, this time considering reviewers based on **US states**.

In [ ]:
# Count the number of reviewers in each country
reviewers_counts = users_loc['user_state'].value_counts().reset_index()
reviewers_counts.columns = ['user_state', 'nb_reviewers']

# Merge reviewer counts with the world map data
world_merge = world_with_US_states.merge(reviewers_counts, how='left', left_on='name', right_on='user_state')

# Fill NaN values (countries without reviewers) with 0
world_merge['nb_reviewers'].fillna(0, inplace=True)
world_merge['user_state'].fillna('Unknown', inplace=True)

# Identify the top 10 countries
top10 = world_merge.sort_values(by='nb_reviewers', ascending=False).head(10)
top10 = top10[['name', 'nb_reviewers']]
top10.index = np.linspace(1, 10, 10).astype(int)
top10.nb_reviewers = top10.nb_reviewers.astype(int)

# Display the Top 10 countries (considering US states) with the most reviewers
print('Top 10 countries (considering US states) with the most reviewers:')
display(top10)

# Plot the choropleth map
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
world_merge.boundary.plot(ax=ax, color='black', linewidth=0.5)
world_merge.plot(column='nb_reviewers', ax=ax, legend=True,
                 norm=LogNorm(vmin=1, vmax=world_merge['nb_reviewers'].max()),
                 legend_kwds={'label': "Number of Reviewers by Country", 'orientation': "vertical"},
                 cmap='Blues')

# Remove the axis
ax.set_title('Distribution of the reviewers in the World')
ax.set_axis_off()
plt.show()


We can see that **the majority of reviewers are concentrated in the USA**. In fact, not only the majority of breweries but also the vast majority of reviewers are located in the USA. This is certainly a crucial factor to take into account in future analyses.

# 3. Global Analysis

## 3.1 Ratings vs Size

We will now analyze if and how the size metric impacts the average rating obtained by the breweries. 
A priori, one might expect the scores obtained by small breweries to be fairly evenly distributed, ranging from very bad to very good, depending on taste, with potentially more polarizing flavours. On the other hand, in the case of large breweries with an international reach, we might expect the scores to be more grouped, reflecting products that are more polished and more complex. 

In [ ]:
df_rating_analysis = df_metrics_macro.copy(deep=True)

# Compute the log of the avg. rating for each breweries
df_rating_analysis['log_avg_rating'] = df_metrics_macro['avg_rating_macro'].apply(lambda x : np.log(x))
df_rating_analysis['log_size_metrics'] = df_metrics_macro['size_metrics_macro'].apply(lambda x : np.log(x))

In [ ]:
plt.figure(figsize=(6, 4))

point_size = 2

# Plot scatter 'size_metrics_macro'
plt.scatter(df_rating_analysis['size_metrics_macro'], df_rating_analysis['avg_rating_macro'], color='cornflowerblue', label='avg. rating', s=point_size)

plt.title('Distribution of Ratings w.r.t. Size and Popularity Metrics')
plt.ylabel('Avg. rating obtained')
plt.xlabel('Metrics')

plt.legend()
plt.tight_layout()
plt.show()

It can be seen that the distribution is highly dispersed for the smaller breweries, sweeping almost the entire range of scores, whereas the average score obtained by the larger breweries seems to be slightly higher and less spread out.

We discretize the size metrics by interval of size 0.05 in order to class the breweries into categories and see the rating trend with respect to the categories.

In [ ]:
# Define the size interval width
interval_width = 0.05

# Calculate the minimum and maximum size values to determine the range
min_size = df_rating_analysis['size_metrics_macro'].min()
max_size = df_rating_analysis['size_metrics_macro'].max()

# Create intervals for size
size_intervals = pd.interval_range(start=min_size, end=max_size, freq=interval_width)

# Assign each row to its corresponding interval
df_rating_analysis['interval'] = pd.cut(df_rating_analysis['size_metrics_macro'], bins=size_intervals)

mean_std_rating_by_size = df_rating_analysis.groupby('interval')['avg_rating_macro'].agg(['mean', 'std']).reset_index()

display(mean_std_rating_by_size)

In [ ]:
# Middle points of the size metric intervals
mean_std_rating_by_size['midpoints'] = mean_std_rating_by_size['interval'].apply(lambda x: x.mid)

plt.figure(figsize=(8, 6))

for i in range(len(mean_std_rating_by_size)):
    if i % 2 == 0:
        plt.axvspan(mean_std_rating_by_size['interval'][i].left, mean_std_rating_by_size['interval'][i].right, color='lightgray', alpha=0.5)
    
plt.scatter(df_rating_analysis['size_metrics_macro'], df_rating_analysis['avg_rating_macro'], color='#AED6F1',label='avg. rating', s=point_size)
plt.errorbar(mean_std_rating_by_size['midpoints'], mean_std_rating_by_size['mean'], yerr=mean_std_rating_by_size['std'], fmt='o-', color='b', ecolor='cornflowerblue', label='mean and std rating for each size segment', capsize=5)
plt.title(f'Average rating by {interval_width:.2f} size categories with Standard deviation')
plt.xlabel('Size metric (segmented)')
plt.ylabel('Avg. rating')

plt.legend()
#plt.grid(True)
plt.tight_layout()
plt.show()

We can observe a fairly linear increase in the mean rating obtained as the size metric increases, as well as a tightening of the standard deviation. According to that, being a larger brewery would therefore tend to increase the chances of obtaining a higher overall rating.

### 3.1.1 Regression analysis

To check whether a linear relationship can be discerned between the size metric and the average rating obtained by the breweries, we run a linear regression on the raw distribution.

In [ ]:
model_continous = smf.ols(formula='avg_rating_macro ~ size_metrics_macro', data=df_rating_analysis).fit()
print(model_continous.summary())

There does seem to be a linear correlation between having a large size index and the average rating obtained by a brewery. However, the $R^2 = 0.065$ value indicates that this regression explains the correlation rather poorly.

If we consider the case where the rating is averaged by metric size, the linear correlation is, as expected, much stronger.

In [ ]:
model_discret = sm.OLS(mean_std_rating_by_size['mean'], mean_std_rating_by_size['midpoints']).fit()
print(model_discret.summary())

However, we must remain cautious with these results, as it is likely that other cofounders, not taken into account, have an impact on the average rating obtained by the breweries. It is therefore likely that the size index does not explain everything.

### 3.1.2 Rating vs Size Categories

Now that the distribution and general trend of the average rating in relation to the size index has been observed, it is interesting to categorize the breweries into 3 different sizes, small, medium and big in order to see if belonging to a of these categories explains the overall score in a statistically significant way.

In [ ]:
# Create lists for means and standard deviations
means, std_devs = helpfn.compute_stats_for_categories(df_rating_analysis,'avg_rating_macro')
labels = ['Small', 'Medium', 'Big']
colors = ['lightgreen', 'lightblue', 'lightcoral']
dark_colors  = ['darkgreen', 'darkblue', 'indianred']
size = [0, 0.3, 0.7, 1.0]
mid = [0.15, 0.5, 0.85]
point_size = 3

plt.figure(figsize=(8, 6))
labels = ['Small breweries', 'Medium breweries', 'Big breweries']
text_positions = zip(mid, [1.5, 1.5, 1.5], labels, dark_colors)

for i in range(len(size)-1):
    plt.axvspan(size[i], size[i+1], color=colors[i], alpha=0.5)
    plt.errorbar(mid[i], means[i], yerr=std_devs[i], fmt='o-', color=dark_colors[i], ecolor=dark_colors[i], label='mean and std rating for '+labels[i], capsize=5, alpha=1.0)
    #plt.scatter(mid[i], means[i], color=dark_colors[i], s=100)  # Adding points at text positions
    for pos in text_positions:
        plt.text(pos[0], pos[1], pos[2], color=pos[3], ha='center', va='center', fontsize=14)

plt.scatter(df_rating_analysis['size_metrics_macro'], df_rating_analysis['avg_rating_macro'], color='darkgray',label='avg. rating distribution', s=point_size)
plt.legend(loc='upper right')
plt.title('Avg. rating w.r.t. Size metrics with mean rating and std. for the size categories')
plt.xlabel('Size metrics')
plt.ylabel('Avg. rating')

plt.tight_layout()
plt.show()

print('Average rating for the small breweries: {:.3g} with std: {:.3g}'.format(means[0], std_devs[0]))
print('Average rating for the medium breweries: {:.3g} with std: {:.3g}'.format(means[1], std_devs[1]))
print('Average rating for the big breweries: {:.3g} with std: {:.3g}'.format(means[2], std_devs[2]))

It can be seen that belonging to a larger category tends to increase the average rating obtained quite significantly. An average rating difference of almost 0.4 can be observed between small and large breweries. However, we must be careful, as the standard deviation remains quite high.

### Is there a difference over the years ?


It is now legitimate to question the temporal robustness of this result. Can the same conclusion be drawn in 2005, 2010 and 2016? To investigate this, the same analysis of the average rating on the 3 size categories over these years is carried out.

In [ ]:
df_temporal = df_metrics.copy(deep=True)

In [ ]:
years_list = [2005, 2010, 2016]
dfs = {} 
means = {}
stds = {}

for year in years_list:
    dfs[f"df_small_breweries_{year}"] = df_temporal[(df_temporal['size_category']=='small') & (df_temporal['date']==year)]
    dfs[f"df_medium_breweries_{year}"] = df_temporal[(df_temporal['size_category']=='medium') & (df_temporal['date']==year)]
    dfs[f"df_big_breweries_{year}"] = df_temporal[(df_temporal['size_category']=='big') & (df_temporal['date']==year)]

    means[f"mean_small_{year}"] = dfs[f'df_small_breweries_{year}']['avg_rating'].mean()
    stds[f"std_small_{year}"] = dfs[f'df_small_breweries_{year}']['avg_rating'].std()

    means[f"mean_medium_{year}"] = dfs[f'df_medium_breweries_{year}']['avg_rating'].mean()
    stds[f"std_medium_{year}"] = dfs[f'df_medium_breweries_{year}']['avg_rating'].std()

    means[f"mean_big_{year}"] = dfs[f'df_big_breweries_{year}']['avg_rating'].mean()
    stds[f"std_big_{year}"] = dfs[f'df_big_breweries_{year}']['avg_rating'].std()


In [ ]:
categories = ['small', 'medium', 'big']
labels = ['Small breweries', 'Medium breweries', 'Big breweries']
colors = ['lightgreen', 'lightblue', 'lightcoral']
dark_colors = ['darkgreen', 'darkblue', 'indianred']
size = [0, 0.3, 0.7, 1.0]
mid = [0.15, 0.5, 0.85]

def plot_on_subplot(ax, j):
    for i in range(len(size) - 1):
        ax.axvspan(size[i], size[i + 1], color=colors[i], alpha=0.5)
        ax.errorbar(mid[i], means[f"mean_{categories[i]}_{years_list[j]}"], yerr=stds[f"std_{categories[i]}_{years_list[j]}"], fmt='o-', color=dark_colors[i],
                    ecolor=dark_colors[i],label='Mean and std rating ' + str(categories[i]),
                    capsize=5, alpha=1.0)
        if j == 0:
            ax.scatter(dfs[f"df_{categories[i]}_breweries_{years_list[j]}"]['size_metrics'],
                    dfs[f"df_{categories[i]}_breweries_{years_list[j]}"]['avg_rating'], color='darkgray',
                   label='rating distribution', s=point_size)
        else:
            ax.scatter(dfs[f"df_{categories[i]}_breweries_{years_list[j]}"]['size_metrics'],
                    dfs[f"df_{categories[i]}_breweries_{years_list[j]}"]['avg_rating'], color='darkgray', s=point_size)
        ax.text(mid[i], 1.7, labels[i], color=dark_colors[i], ha='center', va='center', fontsize=12)

        ax.set_title('Avg. rating distribution ' + str(years_list[j]))
        #ax.set_yscale('log')
        ax.set_ylim([1.5,5])
        ax.set_xlim([0,1])
        ax.set_xlabel('Size metrics')
        ax.set_ylabel('Avg. rating')
        
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # Creating three subplots side by side

for j,ax in enumerate(axs):
    plot_on_subplot(ax,j)
    

plt.legend(loc=(0.4,0.25))
plt.suptitle('Avg. rating distribution w.r.t. ize metric with mean yearly rating and std. for the size categories and for years 2005, 2010, 2016')
plt.tight_layout()
plt.show()

# Print mean and std for each size categories and years
for year in years_list:
    print(f"Mean rating for the small breweries in {year}: {means[f'mean_small_{year}']:.4g} with std: {stds[f'std_small_{year}']:.4g}")
    print(f"Mean rating for the medium breweries in {year}: {means[f'mean_medium_{year}']:.4g} with std: {stds[f'std_medium_{year}']:.4g}")
    print(f"Mean rating for the big breweries in {year}: {means[f'mean_big_{year}']:.4g} with std: {stds[f'std_big_{year}']:.4g}")
    print("\n")

We note that the same conclusion as in section 3.1.2 applies to all years. Belonging to a bigger category would tend to increase the average rating obtained. There is therefore no major temporal evolution in this analysis.

### Hypothesis testing

The final investigation is to test whether there really is a statistically significant difference in terms of average rating when belonging to one size category rather than another. To do this, a **Student's t-test** is performed on the average rating between the *small/medium* and *medium/big* categories.

We perform t-test anlysis at the 0.05 significance level under the hypotesis $H_0$ : *There is no statistically significant difference in average rating between the breweries size categories.* To see if the size category of a brewery impacts the average rate obtained.

In [ ]:
t_statistic, p_value = stats.ttest_ind(df_rating_analysis[df_rating_analysis['size_category']=='small']['avg_rating_macro'],
                                       df_rating_analysis[df_rating_analysis['size_category']=='medium']['avg_rating_macro'], equal_var=False)

# Significance level
alpha = 0.05

print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.8}")

if p_value < alpha:
    print("The difference in avg. rating is statistically significant between small and medium breweries.")
else:
    print("The difference in avg. rating is not statistically significant between small and medium breweries.")

In [ ]:
t_statistic, p_value = stats.ttest_ind(df_rating_analysis[df_rating_analysis['size_category']=='medium']['avg_rating_macro'],
                                       df_rating_analysis[df_rating_analysis['size_category']=='big']['avg_rating_macro'], equal_var=False)

# Significance level
alpha = 0.05

print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.8f}")

if p_value < alpha:
    print("The difference in avg. rating is statistically significant between medium and big breweries.")
else:
    print("The difference in avg. rating is not statistically significant between medium and big breweries.")

We can conclude that there is likely to be a significant (at significance level 0.05) statisticall difference between the small, medium and large breweries in term of rating obtained. Given the plot just on top, being a bigger breweries tend to increase the mean rating obtained and reduce the standard deviation.

## 3.2 Review length vs. brewery size analysis

We do the same analysis but to see if the brewery size metrics impacts the average review length obtained by the breweries

In [ ]:
df_review_len = df_metrics_macro.copy(deep=True)

In [ ]:
# Create lists for means and standard deviations
means_rew_len, std_devs_rew_len = helpfn.compute_stats_for_categories(df_review_len,'avg_len_text_macro')

labels = ['Small', 'Medium', 'Big']
colors = ['lightgreen', 'lightblue', 'lightcoral']
dark_colors  = ['darkgreen', 'darkblue', 'indianred']
size = [0, 0.3, 0.7, 1.0]
mid = [0.15, 0.5, 0.85]

plt.figure(figsize=(8, 6))

labels = ['Small breweries', 'Medium breweries', 'Big breweries']
text_positions = zip(mid, [2500, 2500, 2500], labels, dark_colors)

for i in range(len(size)-1):
    plt.axvspan(size[i], size[i+1], color=colors[i], alpha=0.5)
    plt.errorbar(mid[i], means_rew_len[i], yerr=std_devs_rew_len[i], fmt='o-', color=dark_colors[i], ecolor=dark_colors[i],label='mean and std review len. for '+labels[i], capsize=5, alpha=1.0)
    
    #plt.scatter(mid[i], means[i], color=dark_colors[i], s=100)  # Adding points at text positions
    for pos in text_positions:
        plt.text(pos[0], pos[1], pos[2], color=pos[3], ha='center', va='center', fontsize=14)

plt.scatter(df_review_len['size_metrics_macro'], df_review_len['avg_len_text_macro'], color='darkgray', label='review len distribution', s=point_size)
plt.legend(loc='lower right')
plt.title('Avg. review length w.r.t. Size metric with mean review lenght and std. for the size categories')
plt.yscale('log')
plt.xlabel('Size metric')
plt.ylabel('Avg. review length')
plt.tight_layout()
plt.show()

print('Averge review length for the small breweries: ', means_rew_len[0], 'with std: ', std_devs_rew_len[0])
print('Averge review length for the medium breweries: ', means_rew_len[1], 'with std: ', std_devs_rew_len[1])
print('Averge review length for the big breweries: ', means_rew_len[2], 'with std: ', std_devs_rew_len[2])

### **Hypothesis testing**

We perform t-test anlysis at the 0.05 significance level under the hypotesis $H_0$ : *There is no statistically significant difference in average review length between the breweries size categories.* To see if the size category of a brewery impacts the average review length obtained.

In [ ]:
t_statistic, p_value = stats.ttest_ind(df_review_len[df_review_len['size_category']=='small']['avg_len_text_macro'],
                                       df_review_len[df_review_len['size_category']=='medium']['avg_len_text_macro'], equal_var=False)

# Significance level
alpha = 0.05

print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.8f}")

if p_value < alpha:
    print("The difference in avg. review length is statistically significant between small and medium breweries.")
else:
    print("The difference in avg. review length is not statistically significant between small and medium breweries.")

In [ ]:
t_statistic, p_value = stats.ttest_ind(df_review_len[df_review_len['size_category']=='medium']['avg_len_text_macro'],
                                       df_review_len[df_review_len['size_category']=='big']['avg_len_text_macro'], equal_var=False)

# Significance level
alpha = 0.05

print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.8f}")

if p_value < alpha:
    print("The difference in avg. review length is statistically significant between medium and big breweries.")
else:
    print("The difference in avg. review length is not statistically significant between medium and big breweries.")

## 3.3 Geographical Distribution of Reviewers vs Size

### 3.3.1 Relative distance between Reviewers and Breweries

Our next focus is to **explore the relative distance between the reviewer and the brewery for each review**. This analysis aims to provide insights into **how the popularity of a brewery is distributed globally**.

To achieve this, we begin by calculating the centroid of each country (or state in the case of US). Subsequently, we can compute the distance between the countries/state.

In [ ]:
# Add centroid to world dataset
world_df = world_with_US_states.copy(deep=True)
world_df = world_df.to_crs(3857)
world_df['centroids'] = world_df['geometry'].centroid.to_crs(4326)
world_df = world_df.reset_index(drop=True)
display(world_df.sample(3))

In [ ]:
# Create an empty DataFrame to store distances
distances_df = pd.DataFrame(index=world_df['name'], columns=world_df['name'])

# Calculate distances for each pair of countries
for i in range(len(world_df)):
    for j in range(i + 1, len(world_df)):
        country1 = world_df.loc[i, 'name']
        country2 = world_df.loc[j, 'name']
        lat1, lon1 = gpd.GeoSeries(world_df.loc[i, 'centroids']).y, gpd.GeoSeries(world_df.loc[i, 'centroids']).x
        lat2, lon2 = gpd.GeoSeries(world_df.loc[j, 'centroids']).y, gpd.GeoSeries(world_df.loc[j, 'centroids']).x
        
        # Compute haversine distance
        distance = helpfn.haversine(lat1, lon1, lat2, lon2)
        
        # Fill both entries since the distance is symmetric
        distances_df.at[country1, country2] = distance
        distances_df.at[country2, country1] = distance


In [ ]:
# Merge the brewery/user location and state with reviews
review_brew_user = reviews_filt[['brewery_id','user_id','date']].copy(deep=True)
review_brew_user = pd.merge(review_brew_user,breweries_loc[['brewery_id','brewery_country','brewery_state']],on=['brewery_id'],how='inner')
review_brew_user = pd.merge(review_brew_user,users_loc[['user_id','user_country','user_state']],on=['user_id'],how='inner')
display(review_brew_user.sample(3))

Now that we have a dataframe containing every review, along with the location of the reviewed brewery and the user's residence, we can add the distance computed above in the dataframe.

In [ ]:
rev_distance = review_brew_user.copy(deep=True)
# Create new column with distance between reviewer and the brewery for each review
rev_distance['distance_state'] = rev_distance.apply(lambda row: distances_df.loc[row.brewery_state,row.user_state],axis=1)
# Replace the NaN by 0, (the distance between the same country is set to NaN by defintion (distance(belgium,belgium) = NaN))
rev_distance = rev_distance.fillna(0)
display(rev_distance.sample(3))

To check the consistency of the distance, we use the tool from Google Maps to compute distance. We can see below that our measure is pretty accurate. The computed distance between Belgium and Ohio is 6504 km wheras the tool from Google Maps shows 6518 km.

In [ ]:
rev_distance.iloc[2269445]

<center><img title="Distance between Belgium and Ohio" alt="Distance between Belgium and Ohio" src="images/Be_Ohio.png" width="800"></center>
<center>Distance between Belgium and Ohio (Google Maps)</center>


Let's look at the distribution of the distances

In [ ]:
rev_dist = rev_distance.copy(deep=True)
# Plot a histogram of the distances between reviewer and brewery locations
plt.hist(rev_dist['distance_state'], bins=100, color='Blue')

# Set x and y labels and the title
plt.xlabel('Distance between reviewer location and brewery location [km]')
plt.ylabel('Number of reviews')
plt.title('Reviewers relative distance to breweries')

# Display the plot
plt.show()

The graph showing the distribution of relative distances between reviewers and breweries for each review is quite diversified, with a majority of user reviewing "local" beers (that comes from the same state/country), and then a large proportion of reviews between 0 and 8000km, corresponding notably to intra-USA reviews.

### 3.3.2 Is there a relation between the size and the spatial distribution of the reviewers ?

It's now interesting to look at the relative distance of reviewers for a given brewery based on its **size**. This investigation would make it possible to discern whether a small local brewery receives a majority of reviews from nearby users, or whether a world-renowned brewery receives international reviews, resulting in a higher average distance. Since the vast majority of the reviewers are located in the USA, we decided to restrain this analysis to the USA, meaning that we will only consider the breweries on the american ground. This is is necessary since the non-american breweries will receive most of their reviews from american people and thus their distance will be bigger.

First of all, we need to populate the distance dataframe with information on brewery size.

In [ ]:
# Merge distance and metrics dataframes on 'brewery_id'
metric_distance = pd.merge(rev_dist[['brewery_id', 'distance_state','brewery_country','user_country']],
                          df_metrics_macro[['brewery_id', 'size_metrics_macro', 'popularity_metrics_macro','size_category']],
                          on=['brewery_id'], how='inner')

display(metric_distance.sample(3))

In [ ]:
# Keep only US breweries
USA_metric_dist = metric_distance[metric_distance['brewery_country']=='United States of America']

We will look to the mean distance of the reviewers for each brewery.

In [ ]:
# Group by 'brewery_id' and calculate the mean distance of the reviewers
USA_mean = USA_metric_dist.groupby('brewery_id').agg({
    'brewery_country': 'first', 
    'distance_state': 'mean',
    'user_country': 'first', 
    'size_metrics_macro': 'first',
    'size_category': 'first',
    'popularity_metrics_macro': 'first'
}).reset_index()

In [ ]:
# Create lists for means and standard deviations
means, std_devs = helpfn.compute_stats_for_categories(USA_mean,'distance_state')
labels = ['Small', 'Medium', 'Big']
colors = ['lightgreen', 'lightblue', 'lightcoral']
dark_colors  = ['darkgreen', 'darkblue', 'indianred']
size = [0, 0.3, 0.7, 1.0]
mid = [0.15, 0.5, 0.85]

plt.figure(figsize=(8, 6))
labels = ['Small breweries', 'Medium breweries', 'Big breweries']
text_positions = zip(mid, [5000, 5000, 5000], labels, dark_colors)

for i in range(len(size)-1):
    plt.axvspan(size[i], size[i+1], color=colors[i], alpha=0.5)
    plt.errorbar(mid[i], means[i], yerr=std_devs[i], fmt='o-', color=dark_colors[i], ecolor=dark_colors[i], label='mean and std distance for '+labels[i], capsize=5, alpha=1.0)
    #plt.scatter(mid[i], means[i], color=dark_colors[i], s=100)  # Adding points at text positions
    for pos in text_positions:
        plt.text(pos[0], pos[1], pos[2], color=pos[3], ha='center', va='center', fontsize=14)

plt.scatter(USA_mean['size_metrics_macro'], USA_mean['distance_state'], color='darkgray',label='Breweries', s=3)
plt.legend(loc='upper right')
plt.title('Avg. review distance w.r.t. Size metric with mean rating and std. for the size categories')
plt.xlabel('Size metric')
plt.ylabel('Avg. Distance')
plt.xlim([0,1])


plt.tight_layout()
plt.show()
print('Macro-average of the reviewer distance for the small breweries: ', means[0], 'with std: ', std_devs[0])
print('Macro-average of the reviewer distance for the medium breweries: ',means[1], 'with std: ', std_devs[1])
print('Macro-average of the reviewer distance for the big breweries: ', means[2], 'with std: ', std_devs[2])

The resulting plot is very interesting. We can see that a big portion of the small breweries has a mean distance to their reviewer close to zero. Meaning that their reviewers are living in the same state. This is not the case for bigger breweries, that seem to be reviewed by a more global panel of reviewers. It would be interesting to see now if this results holds over the years.

In [ ]:
# Merge distance and metrics dataframes on 'brewery_id'
yearly_metric_distance = pd.merge(rev_dist[['brewery_id', 'distance_state','brewery_country','user_country','date']],
                          df_metrics[['brewery_id','date', 'size_metrics','size_category']],
                          on=['brewery_id','date'], how='inner')
# Keep only Us breweries
USA_yearly_metric_distance = yearly_metric_distance[yearly_metric_distance['brewery_country']=='United States of America']
display(yearly_metric_distance.sample(3))

In [ ]:
# Group by 'brewery_id' and calculate the mean distance of the reviewers
years = [2005, 2010, 2016]
dfs = []
for year in years:
    dfs.append(USA_yearly_metric_distance[USA_yearly_metric_distance['date']==year].groupby('brewery_id').agg({
        'brewery_country': 'first', 
        'distance_state': 'mean',
        'user_country': 'first', 
        'size_metrics': 'first',
        'size_category': 'first',
        'date': 'first'
    }).reset_index())

In [ ]:
labels = ['Small', 'Medium', 'Big']
colors = ['lightgreen', 'lightblue', 'lightcoral']
dark_colors  = ['darkgreen', 'darkblue', 'indianred']
size = [0, 0.3, 0.7, 1.0]
mid = [0.15, 0.5, 0.85]

fig, axs = plt.subplots(1, 3, figsize=(18, 6))
labels = ['Small breweries', 'Medium breweries', 'Big breweries']
text_positions = zip(mid, [5200, 5200, 5200], labels, dark_colors)
for j,df in enumerate(dfs):
    # Create lists for means and standard deviations
    means, std_devs = helpfn.compute_stats_for_categories(dfs[j],'distance_state')

    for i in range(len(size)-1):

        axs[j].axvspan(size[i], size[i+1], color=colors[i], alpha=0.5)
        axs[j].errorbar(mid[i], means[i], yerr=std_devs[i], fmt='o-', color=dark_colors[i], ecolor=dark_colors[i], label='mean and std distance for '+labels[i], capsize=5, alpha=1.0)
        for p,pos in enumerate(text_positions):
            axs[j].text(pos[0], pos[1], pos[2], color=pos[3], ha='center', va='center', fontsize=11)

    axs[j].scatter(df['size_metrics'], df['distance_state'], color='darkgray',label='Breweries', s=3)
    text_positions = zip(mid, [5200, 5200, 5200], labels, dark_colors)
    axs[j].set_title(f'Avg. review distance w.r.t. Size metric \n {years[j]}')
    axs[j].set_xlabel('Size metric')
    axs[j].set_ylabel('Avg. Distance')
    axs[j].set_xlim([0,1])
    axs[j].set_ylim([-500,7500])
    axs[j].grid(True,axis='y',linestyle=':')

axs[2].legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
brew2002 = df_metrics.copy(deep=True)
brew2017 = df_metrics.copy(deep=True)
brew2002 = brew2002[brew2002['date']==2002]
brew2017 = brew2017[brew2017['date']==2017]
big2017 = brew2017[brew2017['size_category']=='big']
small2002 = brew2002[brew2002['size_category']=='small']
small2big = pd.merge(small2002,big2017,on=['brewery_id'], how='inner')
big2002 = brew2002[brew2002['size_category']=='big']
small2002 = pd.merge(small2002['brewery_id'],df_metrics[['brewery_id','date','size_metrics']],on=['brewery_id'], how='inner')
big2002 = pd.merge(big2002['brewery_id'],df_metrics[['brewery_id','date','size_metrics']],on=['brewery_id'], how='inner')
display(small2big)

In [ ]:
# Create a line plot using seaborn
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
sns.lineplot(x='date', y='size_metrics', hue='brewery_id', data=small2002, marker='o', ax=axs[0])
axs[0].set_title('Evolution of Breweries that were small in 2002')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Size Metrics')
#axs[0].legend(title='Brewery ID')
sns.lineplot(x='date', y='size_metrics', hue='brewery_id', data=big2002, marker='o', ax=axs[1])
axs[1].set_title('Evolution of Breweries that were big in 2002')
axs[1].set_xlabel('Year')
axs[1].set_ylabel('Size Metrics')
#axs[1].legend(title='Brewery ID')

# Show the plot
plt.show()

In [ ]:
Example = df_metrics[df_metrics['brewery_id']==1199].copy(deep=True)
USA_dist = USA_yearly_metric_distance[['brewery_id','date','distance_state']][USA_yearly_metric_distance['brewery_id']==1199].copy(deep=True)

USA_dist = USA_dist.groupby('date').agg({
        'brewery_id': 'first',
        'distance_state': 'mean'
    }).reset_index()

Example = pd.merge(Example,breweries_loc[['brewery_id','brewery_name']],on=['brewery_id'], how='inner')
Example = pd.merge(Example,USA_dist[['brewery_id','date','distance_state']], on=['brewery_id','date'], how='inner')
display(Example)